<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/neo4jdocs/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium==4.7.2 transformers==4.25.1 sentence-transformers===2.2.2 graphdatascience

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 22.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 50.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.7/183.7 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.1/757.1 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 74.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 693.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 40.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
#!apt-get update
#!apt install chromium-chromedriver
#!apt install -y xvfb

!pip install undetected-chromedriver==3.1.7
!pip install PyVirtualDisplay

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 771.6 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.4 MB/s eta 0:00:00
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.1.7-py3-none-any.whl size=45427 sha256=65da74c863193515ab1c62bffb2ad6f14bb2897c7466601eafd61718789d0b2b
  Stored in directory: /home/jupyter/.cache/pip/wheels/a9/03/11/ad845fc2d562b9194cc8940e980026205fc8c7168d9de0b254
Successfully built undetected-chromedriver


In [7]:
# Only run this if you use Google Colab

!zip -j /content/chromedriver_linux64.zip /usr/bin/chromedriver
# replace python3.8 with your own version of python in case it's not the same
patcher_src = (
    "/usr/local/lib/python3.8/dist-packages/undetected_chromedriver/patcher.py"
)
with open(patcher_src, "r") as f:
    contents = f.read()
    contents = contents.replace(
        "return urlretrieve(u)[0]",
        "return urlretrieve('file:///content/chromedriver_linux64.zip',"
        "filename='/tmp/chromedriver_linux64.zip')[0]",
    )
with open(patcher_src, "w") as f:
    f.write(contents)

updating: chromedriver (deflated 51%)


In [8]:
# Only run this if you use Google Colab
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [1]:
import undetected_chromedriver.v2 as uc
from pyvirtualdisplay import Display

display = Display(visible=0, size=(800, 600))
display.start()

options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
wd = uc.Chrome(options=options)

In [2]:
def extract_text_by_class(c):
    global wd
    try:
        content = wd.find_element(By.CLASS_NAME, c)
        return content.text
    except:
        return ""


def extract_links_by_xpath(xpath):
    global wd
    links = set()
    try:
        a_elems = wd.find_elements(By.XPATH, xpath)
        for elem in a_elems:
            link = elem.get_attribute("href")
            if link == "javascript:void(0)":
                continue
            # Remove links to images and various files
            if (
                link.endswith(".png")
                or link.endswith(".json")
                or link.endswith(".txt")
                or link.endswith(".svg")
                or link.endswith(".ipynb")
                or link.endswith(".jpg")
                or link.endswith(".pdf")
                or link.endswith(".mp4")
                or "mailto" in link
                or len(link) > 300
            ):
                continue
            # Remove anchors
            link = link.split("#")[0]
            # Remove parameters
            link = link.split("?")[0]
            # Remove trailing forward slash
            link = link.rstrip("/")
            links.add(link)
        return list(links)
    except:
        return []

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("yanekyuk/bert-uncased-keyword-extractor")
model = AutoModelForTokenClassification.from_pretrained(
    "yanekyuk/bert-uncased-keyword-extractor"
)

nlp = pipeline("ner", model=model, tokenizer=tokenizer)


def extract_keywords(text):
    """
    Extract keywords and construct them back from tokens
    """
    result = list()
    keyword = ""
    for token in nlp(text):
        if token["entity"] == "I-KEY":
            keyword += (
                token["word"][2:]
                if token["word"].startswith("##")
                else f" {token['word']}"
            )
        else:
            if keyword:
                result.append(keyword)
            keyword = token["word"]
    # Add the last keyword
    result.append(keyword)
    return list(set(result))

In [4]:
extract_keywords(
    """
Broadcom agreed to acquire cloud computing company VMware in a $61 billion (€57bn) cash-and stock deal.
"""
)

['cloud computing', 'broadcom', 'vmware']

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


def generate_embeddings(text):
    embeddings = model.encode(text)
    return [float(x) for x in embeddings.tolist()]

In [6]:
generate_embeddings(
    """
Web APIs are a huge opportunity to access and integrate data from any sources with your graph. Most of them provide the data in JSON format.

The Load JSON procedures retrieve data from URLs or maps and turn it into map value(s) for Cypher to consume. Cypher has support for deconstructing nested documents with dot syntax, slices, UNWIND etc. so it is easy to turn nested data into graphs.

Sources with multiple JSON objects (JSONL,JSON Lines) in a stream, like the streaming Twitter format or the Yelp Kaggle dataset, are also supported,
"""
)

[-0.06911397725343704,
 0.05224502459168434,
 -0.00720175402238965,
 -0.015374758280813694,
 0.0039060525596141815,
 -0.05042458698153496,
 -0.10675830394029617,
 0.062257084995508194,
 0.00325472722761333,
 0.013045601546764374,
 0.00743217533454299,
 0.0024386586155742407,
 0.07324245572090149,
 0.06017129123210907,
 0.13454288244247437,
 0.03666399419307709,
 -0.007088605314493179,
 0.030180761590600014,
 0.026259899139404297,
 -0.147831529378891,
 0.01147543266415596,
 0.0016846602084115148,
 -0.04652785509824753,
 -0.03765551745891571,
 0.04880434647202492,
 0.08559443056583405,
 -0.03026597388088703,
 0.05212768539786339,
 0.04567540064454079,
 0.06373853981494904,
 -0.02977653034031391,
 -0.010671362280845642,
 -0.05804385617375374,
 0.058136411011219025,
 -0.09820529818534851,
 0.009667888283729553,
 0.035300180315971375,
 0.009530092589557171,
 0.013390504755079746,
 0.05266077071428299,
 0.042114611715078354,
 0.012197610922157764,
 -0.04744803532958031,
 0.018271852284669876

In [ ]:
from selenium.webdriver.common.by import By

entry_url = "https://neo4j.com/docs"
data = dict()
visit_list = [entry_url]
already_visited = []

while visit_list:
    # Visit the URL
    current_url = visit_list.pop()
    if current_url in already_visited:
        continue
    print(current_url)
    try:
        wd.get(current_url)
    except:
        print(f"Couldn't open {current_url}")
        already_visited.append(current_url)
        display = Display(visible=0, size=(800, 600))
        display.start()

        options = uc.ChromeOptions()
        options.add_argument("--no-sandbox")
        wd = uc.Chrome(options=options)
        continue
    # If redirect
    try:
        actual_url = wd.current_url.rstrip("/").split("#")[0].split("?")[0]
        if actual_url != current_url:
            # Store the redirect information
            data[current_url] = {
                "links": [],
                "text": None,
                "embeddings": [],
                "keywords": [],
                "redirects": [actual_url],
            }
            already_visited.append(current_url)
            # Content is being extracted from the actual url
            current_url = actual_url
    except:
        pass
    # Extract text from the content div
    text = extract_text_by_class("content")
    # If nothing is found, try article div
    if not text:
        text = extract_text_by_class("article")
    # If nothing is found, try page div
    if not text:
        text = extract_text_by_class("page")
    if not text:
        text = extract_text_by_class("single-user-story")
    # Check if 404
    try:
        if "Sorry, page not found" in wd.find_element(By.TAG_NAME, "body").text:
            text = "404"
    except:
        pass

    # Generate paragraph embedding & extract keywords
    if text:
        embeddings = generate_embeddings(text)
        keywords = extract_keywords(text)
    else:
        embeddings = []
        keywords = []

    # Extract links from the content div
    links = extract_links_by_xpath("//div[@class='content']//a[@href]")
    # If nothing is found, try article div
    if not links:
        links = extract_links_by_xpath("//article[@class='article']//a[@href]")
    if not links:
        links = extract_links_by_xpath("//article//a[@href]")

    # if not links and not text:
    #    print(f"Couldn't retrieve the data from {current_url}")
    # Store page information
    data[current_url] = {
        "links": [l for l in links if l != current_url],
        "text": text,
        "embeddings": embeddings,
        "keywords": keywords,
        "redirects": [],
    }
    # Crawling information
    already_visited.append(current_url)
    # Don't leave neo4j.com while crawling
    # and avoid community and sandbox
    visit_list.extend(
        [
            l
            for l in list(links)
            if ("neo4j.com" in l)
            and (not l in already_visited)
            and (not "community.neo4j.com" in l)
            and (not "sandbox.neo4j.com" in l)
        ]
    )

https://neo4j.com/docs
https://neo4j.com/docs/drivers-apis
https://neo4j.com/docs/kafka
https://neo4j.com/docs/kafka/architecture
https://neo4j.com/docs/kafka/neo4j-cluster-connect
https://neo4j.com/docs/java-manual/current
https://neo4j.com/docs/java-manual/current/terminology
https://neo4j.com/docs/java-manual/5/session-api
https://neo4j.com/docs/operations-manual/5/manage-databases/introduction
https://neo4j.com/docs/operations-manual/5/authentication-authorization/access-control
https://neo4j.com/docs/operations-manual/5/authentication-authorization/ldap-integration
https://neo4j.com/docs/operations-manual/5/reference/configuration-settings
https://neo4j.com/docs/operations-manual/5/configuration/file-locations
https://neo4j.com/docs/operations-manual/5/configuration/neo4j-conf
https://neo4j.com/docs/operations-manual/5/configuration/dynamic-settings
https://neo4j.com/docs/operations-manual/5/reference/dynamic-settings-reference
https://neo4j.com/docs/operations-manual/5/reference/

In [ ]:
import json

j = json.dumps(data)
# open file for writing, "w"
f = open("neo4j_docs.json", "w")

# write json object to file
f.write(j)

# close file
f.close()

In [ ]:
# Import to Neo4j
from graphdatascience import GraphDataScience

host = "bolt://44.200.120.93:7687"
user = "neo4j"
password = "customs-spans-misalinements"

gds = GraphDataScience(host, auth=(user, password))

gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (p:Page) REQUIRE p.url IS UNIQUE;
"""
)

gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (k:Keyword) REQUIRE k.name IS UNIQUE;
"""
)

In [ ]:
import_query = """

UNWIND $data AS row
MERGE (p:Page {url:row.url})
SET p.embedding = row.embedding,
    p.has_text = row.has_text,
    p.is_404 = row.is_404
FOREACH (l in row.links    | MERGE (p1:Page {url:l}) MERGE (p)-[:LINKS_TO]->(p1))
FOREACH (k in row.keywords | MERGE (k1:Keyword {name:k}) MERGE (p)-[:HAS_KEYWORD]->(k1))
FOREACH (r in row.redirects| MERGE (r1:Page {url:r}) MERGE (p)-[:REDIRECTS]->(r1))

"""
x = 1
params = []
for key in data:
    params.append(
        {
            "url": key,
            "embedding": data[key]["embeddings"],
            "keywords": data[key]["keywords"],
            "links": data[key]["links"],
            "has_text": True if data[key]["text"] else False,
            "is_404": True if data[key]["text"] == "404" else False,
            "redirects": data[key]["redirects"],
        }
    )
    # Batch per 500
    if len(params) == 500:
        gds.run_cypher(import_query, {"data": params})
        params = []
        # Logging
        print(f"Importing {x} batch")
        x += 1

# Import the remainder
gds.run_cypher(import_query, {"data": params})